In [1]:
!git clone https://github.com/mxbastidasr/DAWN_WACV2020.git

Cloning into 'DAWN_WACV2020'...


In [2]:
%cd /content/DAWN_WACV2020

/content/DAWN_WACV2020


In [3]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset  = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

print("Train:", len(trainset), "images")
print("Test:", len(testset), "images")


100%|██████████| 169M/169M [00:01<00:00, 87.4MB/s]


Train: 50000 images
Test: 10000 images


In [88]:
!python train.py --epochs 150 -b 64 --lr 0.03 --name dwnn_cifar-100_32 \
--lrdecay 150 225 --database cifar-100 --tempdir ~/tmp dawn \
--regu_detail 0.1 --regu_approx 0.1 --levels 3 --first_conv 128

Ignore tensorboard logger
Launch...
['train.py', '--epochs', '150', '-b', '64', '--lr', '0.03', '--name', 'dwnn_cifar-100_32', '--lrdecay', '150', '225', '--database', 'cifar-100', '--tempdir', '/root/tmp', 'dawn', '--regu_detail', '0.1', '--regu_approx', '0.1', '--levels', '3', '--first_conv', '128']
DAWN:
- first conv: 128
- image size: 32
- nb levels : 3
- levels U/P: [2, 1]
- channels:  3
Final channel: 1280
Final size   : 4
Number of model parameters            : 2,646,500
Number of *trainable* model parameters: 2,646,500
Learning rate:  0.03
Epoch: [0][0/782]	Time 0.890 (0.890)	Loss (Class) 5.4526 (5.4526)	Loss (Regu) 0.8434 (0.8434)	Prec@1 0.000 (0.000)	Prec@5 7.812 (7.812)
Epoch: [0][50/782]	Time 0.024 (0.043)	Loss (Class) 4.8828 (5.2249)	Loss (Regu) 0.6114 (0.6912)	Prec@1 4.688 (3.615)	Prec@5 20.312 (14.124)
Epoch: [0][100/782]	Time 0.032 (0.034)	Loss (Class) 4.6778 (4.9706)	Loss (Regu) 0.5631 (0.6387)	Prec@1 4.688 (5.167)	Prec@5 20.312 (18.502)
Epoch: [0][150/782]	Time 0.023 

In [40]:
import requests

url = "https://www.csc.kth.se/cvap/databases/kth-tips/kth-tips2-b_col_200x200.tar"
out_path = "/content/DAWN_WACV2020/data/kth-tips2-b_col_200x200.tar"

with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:  # filter out keep-alive chunks
                f.write(chunk)

print("Downloaded to", out_path)


Downloaded to /content/DAWN_WACV2020/data/kth-tips2-b_col_200x200.tar


In [43]:
import tarfile
tar_path = "/content/DAWN_WACV2020/data/kth-tips2-b_col_200x200.tar"
extract_root = "/content/DAWN_WACV2020/data/KTH-TIPS2-b"  # choose where to extract

with tarfile.open(tar_path, "r:") as tar:
    tar.extractall(path=extract_root)

print("Extracted to", extract_root)


/tmp/ipython-input-856776700.py:6: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_root)


Extracted to /content/DAWN_WACV2020/data/KTH-TIPS2-b


In [46]:
import os, shutil, random

src_root = extract_root  # where images currently are
kth_data_dir = "/content/DAWN_WACV2020/data"
target_root = os.path.join(kth_data_dir, "KTH-TIPS2-b3")

train_dir = os.path.join(target_root, "Train")
test_dir  = os.path.join(target_root, "Test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Example: random 80/20 split over all images
all_imgs = []
for root, _, files in os.walk(src_root):
    for f in files:
        if f.lower().endswith((".jpg", ".jpeg", ".png", ".tif", ".bmp")):
            all_imgs.append(os.path.join(root, f))

random.shuffle(all_imgs)
split = int(0.8 * len(all_imgs))
train_imgs = all_imgs[:split]
test_imgs  = all_imgs[split:]

for p in train_imgs:
    rel = os.path.relpath(p, src_root)
    dst = os.path.join(train_dir, rel)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy2(p, dst)

for p in test_imgs:
    rel = os.path.relpath(p, src_root)
    dst = os.path.join(test_dir, rel)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy2(p, dst)

print("Train:", len(train_imgs), "Test:", len(test_imgs))


Train: 3801 Test: 951


In [49]:
# set env var in this shell
!export KTH_DATA_DIR=/content/DAWN_WACV2020/data

# now run the exact KTH command from the README
!python train.py --epochs 90 -b 16 --lr 0.03 --name dwnn_kth_3_dwnn_l5_16 --lrdecay 30 60 --database kth --traindir $KTH_DATA_DIR/KTH-TIPS2-b3/Test/ --valdir $KTH_DATA_DIR/KTH-TIPS2-b3/Train/ dawn --levels 5 --first_conv 16 --regu_details 0.1 --regu_approx 0.1

Ignore tensorboard logger
Launch...
['train.py', '--epochs', '90', '-b', '16', '--lr', '0.03', '--name', 'dwnn_kth_3_dwnn_l5_16', '--lrdecay', '30', '60', '--database', 'kth', '--traindir', '/content/DAWN_WACV2020/data/KTH-TIPS2-b3/Test/', '--valdir', '/content/DAWN_WACV2020/data/KTH-TIPS2-b3/Train/', 'dawn', '--levels', '5', '--first_conv', '16', '--regu_details', '0.1', '--regu_approx', '0.1']
DAWN:
- first conv: 16
- image size: 224
- nb levels : 5
- levels U/P: [2, 1]
- channels:  3
Final channel: 256
Final size   : 7
Number of model parameters            : 68,667
Number of *trainable* model parameters: 68,667
Learning rate:  0.03
Epoch: [0][0/60]	Time 1.144 (1.144)	Loss (Class) 4.5820 (4.5820)	Loss (Regu) 1.4635 (1.4635)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][50/60]	Time 0.049 (0.062)	Loss (Class) 0.5269 (0.9610)	Loss (Regu) 0.5201 (0.8965)	Prec@1 100.000 (98.039)	Prec@5 100.000 (98.039)
Test: [0/238]	Time 0.179 (0.179)	Loss (Class) 0.6388 (0.6388)	Loss (Regu) 0.6309

In [79]:
!python eval.py \
--database kth \
--num_clases 11 \
--batch-size 16 \
--traindir /content/DAWN_WACV2020/data/KTH-TIPS2-b3/Train \
--valdir /content/DAWN_WACV2020/data/KTH-TIPS2-b3/Test \
--name dwnn_kth_3_dwnn_l5_16 \
--eval_only \
--eval_ckpt /content/DAWN_WACV2020/runs/dwnn_kth_3_dwnn_l5_16/model_best.pth.tar \
dawn \
--levels 5 \
--first_conv 16 \
--regu_details 0.1 \
--regu_approx 0.1


Ignore tensorboard logger
Eval-only script launched
Args: ['eval.py', '--database', 'kth', '--num_clases', '11', '--batch-size', '16', '--traindir', '/content/DAWN_WACV2020/data/KTH-TIPS2-b3/Train', '--valdir', '/content/DAWN_WACV2020/data/KTH-TIPS2-b3/Test', '--name', 'dwnn_kth_3_dwnn_l5_16', '--eval_only', '--eval_ckpt', '/content/DAWN_WACV2020/runs/dwnn_kth_3_dwnn_l5_16/model_best.pth.tar', 'dawn', '--levels', '5', '--first_conv', '16', '--regu_details', '0.1', '--regu_approx', '0.1']
DAWN:
- first conv: 16
- image size: 224
- nb levels : 5
- levels U/P: [2, 1]
- channels:  3
Final channel: 256
Final size   : 7
Number of model parameters: 68667
Number of trainable parameters: 68667
=> loading checkpoint '/content/DAWN_WACV2020/runs/dwnn_kth_3_dwnn_l5_16/model_best.pth.tar'
=> loaded checkpoint '/content/DAWN_WACV2020/runs/dwnn_kth_3_dwnn_l5_16/model_best.pth.tar' (epoch 1, best_prec1=100.000)
Test: [0/60]	Time 0.733 (0.733)	Loss (Class) 0.4108 (0.4108)	Loss (Regu) 0.4022 (0.4022)	Pr

In [89]:
!python eval.py \
--database cifar-100 \
--batch-size 64 \
--name dawn_cifar100_eval \
--eval_only \
--eval_ckpt /content/DAWN_WACV2020/runs/dwnn_cifar-100_32/model_best.pth.tar \
dawn \
--levels 3 \
--first_conv 128 \
--regu_details 0.1 \
--regu_approx 0.1


Ignore tensorboard logger
Eval-only script launched
Args: ['eval.py', '--database', 'cifar-100', '--batch-size', '64', '--name', 'dawn_cifar100_eval', '--eval_only', '--eval_ckpt', '/content/DAWN_WACV2020/runs/dwnn_cifar-100_32/model_best.pth.tar', 'dawn', '--levels', '3', '--first_conv', '128', '--regu_details', '0.1', '--regu_approx', '0.1']
DAWN:
- first conv: 128
- image size: 32
- nb levels : 3
- levels U/P: [2, 1]
- channels:  3
Final channel: 1280
Final size   : 4
Number of model parameters: 2646500
Number of trainable parameters: 2646500
=> loading checkpoint '/content/DAWN_WACV2020/runs/dwnn_cifar-100_32/model_best.pth.tar'
=> loaded checkpoint '/content/DAWN_WACV2020/runs/dwnn_cifar-100_32/model_best.pth.tar' (epoch 136, best_prec1=66.040)
Test: [0/157]	Time 0.597 (0.597)	Loss (Class) 1.4193 (1.4193)	Loss (Regu) 0.2103 (0.2103)	Prec@1 67.188 (67.188)	Prec@5 92.188 (92.188)
Test: [50/157]	Time 0.008 (0.020)	Loss (Class) 1.6353 (1.6657)	Loss (Regu) 0.2185 (0.2159)	Prec@1 68.750

In [72]:
checkpoint_path="/content/DAWN_WACV2020/runs/dwnn_cifar-100_32/model_best.pth.tar"

checkpoint = torch.load(checkpoint_path)
print(checkpoint.keys())
# Common keys: 'epoch', 'state_dict', 'optimizer', 'best_prec1', etc


dict_keys(['epoch', 'state_dict', 'best_prec1'])


In [75]:
import models.dawn as dawn

NUM_CLASS = 100
first_conv = 128
number_levels = 3
regu_details = 0.1
regu_approx = 0.1
kernel_size = 3  # default if unchanged
classifier = 'mode1'  # default
no_bootleneck = False
share_weights = False
simple_lifting = False
haar_wavelet = False
COLOR = True  # Because CIFAR-100 is color images

model = dawn.DAWN(
    NUM_CLASS,
    big_input=True,     # CIFAR input size 32 may count as small or large input depending on your code (check INPUT_SIZE)
    first_conv=first_conv,
    number_levels=number_levels,
    kernel_size=kernel_size,
    no_bootleneck=no_bootleneck,
    classifier=classifier,
    share_weights=share_weights,
    simple_lifting=simple_lifting,
    COLOR=COLOR,
    regu_details=regu_details,
    regu_approx=regu_approx,
    haar_wavelet=haar_wavelet
)


DAWN:
- first conv: 128
- image size: 224
- nb levels : 3
- levels U/P: [2, 1]
- channels:  3
Final channel: 1280
Final size   : 28


In [76]:
state_dict = checkpoint['state_dict']
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)


<All keys matched successfully>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/myfolder /content/drive/MyDrive/


In [91]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [92]:
!cp -r /content/DAWN_WACV2020 /content/drive/MyDrive/ECE_251C